Michał Liss
Marceli Sokólski
Piotr Krzystanek

# Instalacja pig

W kontenerze hadoopa zainstalowany został pig za pomocą nastepujących komend w Dockerfile:

```Dockerfile
RUN wget https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
RUN tar -xf pig-0.17.0.tar.gz
RUN cd pig-0.17.0
RUN mv pig-0.17.0 /usr/local/pig
ENV PATH="${PATH}:/usr/local/pig/bin"
```

# Instalacja hive

Dodane zostały dodatkowe kontenery:
```yml
  hive-server:
    image: przydasie99/pwr-hadoop-hive-server:amd64
    build:
      context: hive-europe
      dockerfile: Dockerfile
    container_name: hive-server
    volumes:
      - ../master_volume:/data/master_volume
    env_file:
      - ./hadoop.env
    environment:
      HIVE_CORE_CONF_javax_jdo_option_ConnectionURL: "jdbc:postgresql://hive-metastore/metastore"
      SERVICE_PRECONDITION: "hive-metastore:9083"
    depends_on:
      - hive-metastore
    ports:
      - "10000:10000"
```
```yml
  hive-metastore:
    image: przydasie99/pwr-hadoop-hive-server:amd64
    build:
      context: hive-europe
      dockerfile: Dockerfile
    container_name: hive-metastore
    env_file:
      - ./hadoop.env
    command: /opt/hive/bin/hive --service metastore
    environment:
      SERVICE_PRECONDITION: "namenode:9000 namenode:9870 datanode:9864 hive-metastore-postgresql:5432"
    depends_on:
      - hive-metastore-postgresql
    ports:
      - "9083:9083"
```
```yml
  hive-metastore-postgresql:
    build:
      context: hive-metastore
      dockerfile: Dockerfile
    image: przydasie99/pwr-hadoop-hive-postgres:amd64
    container_name: hive-metastore-postgresql
    ports:
      - '5433:5432'
```

# Installation



In [1]:
import docker
import uuid
import paramiko
import os
from timeit import default_timer as timer
from dataclasses import dataclass
import re
import requests
import statistics
import pandas as pd
import numpy as np

In [2]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def run_in_hive(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("hive-server", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"bash -c '. /env_var_path.sh && {command}'")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

Create a hive database and schema for the covid table.

In [11]:
run_in_hive("hive -f /data/master_volume/hive_scripts/covid_table.hql")

([],
 ['SLF4J: Class path contains multiple SLF4J bindings.\n',
  'SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: Found binding in [jar:file:/opt/hadoop-3.3.1/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\n',
  'SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n',
  'Hive Session ID = 6de4d833-7859-4c81-ad67-3fee75c161f7\n',
  '\n',
  'Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n',
  'Hive Session ID = 5d1a7066-2ffb-4b51-959c-4accd14cc561\n',
  'OK\n',
  'Time taken: 0.618 seconds\n',
  'OK\n',
  'Time taken: 0.018 seconds\n',
  'OK\n',
  'Time taken: 0.072 seconds\n',
  'OK\n',
  'Time taken: 0.238 seconds\n',
  'OK\n',
  'Time taken: 0.04 seconds\n',
  'OK\n',
  'Time taken: 0.039 secon

In [12]:
run_in_hive("hive -f /data/master_volume/hive_scripts/covid_load_to_hive.hql")

([],
 ['SLF4J: Class path contains multiple SLF4J bindings.\n',
  'SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: Found binding in [jar:file:/opt/hadoop-3.3.1/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\n',
  'SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n',
  'Hive Session ID = 60a425a0-43cb-41f6-8a3a-7d981ba34acc\n',
  '\n',
  'Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n',
  'Hive Session ID = a607fbba-bbb9-494b-9a79-14a65fdd0c4b\n',
  'OK\n',
  'Time taken: 0.617 seconds\n',
  'OK\n',
  'Time taken: 0.069 seconds\n',
  'OK\n',
  'Time taken: 0.275 seconds\n',
  'Loading data to table coviddb.covid_raw\n',
  'OK\n',
  'Time taken: 8.639 seconds\n',
  'Query ID = root_202306021703

Process the data and transform into another table.

In [14]:
start = timer()
results = run_in_hive("hive -f /data/master_volume/hive_scripts/covid_data.hql")
stop = timer()

print(results)
print(f"HIVE JOB TOOK: {stop - start:.2f}s")

([], ['SLF4J: Class path contains multiple SLF4J bindings.\n', 'SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n', 'SLF4J: Found binding in [jar:file:/opt/hadoop-3.3.1/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n', 'SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\n', 'SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n', 'Hive Session ID = 2daaca43-fc28-4fe2-870f-1d06745eaa2a\n', '\n', 'Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n', 'Hive Session ID = 0392f174-1f69-43ee-b53d-d2d17f34e228\n', 'OK\n', 'Time taken: 0.596 seconds\n', 'Query ID = root_20230602170438_af97bea8-73ee-4ed0-943c-1691d5d40dc9\n', 'Total jobs = 1\n', 'Launching Job 1 out of 1\n', 'Number of reduce tasks not specified. Estimated from input data size: 1\n', 'In order to change the aver

Compare the results of the new file with the original.

In [15]:
run_in_master(f"hdfs dfs -cat /user/hive/warehouse/coviddb.db/covid/000000_0")

(['2020-01-03,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-04,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-05,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-06,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-07,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-08,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-09,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-10,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-11,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-12,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-13,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-14,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-15,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-16,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-17,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-18,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-19,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-20,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-21,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-22,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-23,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-24,Afghanistan,\\N,0,\\N,0,0\n',
  '2020-01-25,Afghanistan,\\N,0,

In [16]:
run_in_master(f"hdfs dfs -cat /user/hive/warehouse/coviddb.db/calculated/000000_0")

(['2023-04-19,Zimbabwe,264613,0,5684,0,0,164.6863\n',
  '2023-04-18,Zimbabwe,264613,0,5684,0,0,164.6863\n',
  '2023-04-17,Zimbabwe,264613,3,5684,0,0,164.6863\n',
  '2023-04-16,Zimbabwe,264610,3,5684,0,0,164.6863\n',
  '2023-04-15,Zimbabwe,264607,10,5684,0,0,164.6863\n',
  '2023-04-14,Zimbabwe,264597,4,5684,0,0,164.6863\n',
  '2023-04-13,Zimbabwe,264593,2,5684,0,0,164.6863\n',
  '2023-04-12,Zimbabwe,264591,5,5684,0,0,164.6863\n',
  '2023-04-11,Zimbabwe,264586,2,5684,0,0,164.6863\n',
  '2023-04-10,Zimbabwe,264584,2,5684,0,0,164.6863\n',
  '2023-04-09,Zimbabwe,264582,4,5684,0,0,164.6863\n',
  '2023-04-08,Zimbabwe,264578,2,5684,1,0,164.6863\n',
  '2023-04-07,Zimbabwe,264576,5,5683,1,0,164.6863\n',
  '2023-04-06,Zimbabwe,264571,5,5682,1,0,164.6863\n',
  '2023-04-05,Zimbabwe,264566,4,5681,1,0,164.6863\n',
  '2023-04-04,Zimbabwe,264562,7,5680,\\N,0,164.6863\n',
  '2023-04-03,Zimbabwe,264555,6,5681,0,0,164.6863\n',
  '2023-04-02,Zimbabwe,264549,0,5681,0,0,164.6863\n',
  '2023-04-01,Zimbabwe,26

Compare with Pig

In [17]:
start = timer()
results = run_in_master("pig -x mapreduce /data/master_volume/pig_scripts/test.pig")
stop = timer()

print(results)
print(f"PIG JOB TOOK: {stop - start:.2f}s")

([], ['2023-06-02 17:20:56,455 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL\n', '2023-06-02 17:20:56,455 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE\n', '2023-06-02 17:20:56,455 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType\n', '2023-06-02 17:20:56,480 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58\n', '2023-06-02 17:20:56,480 [main] INFO  org.apache.pig.Main - Logging error messages to: /app/pig_1685726456476.log\n', '2023-06-02 17:20:56,618 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found\n', '2023-06-02 17:20:56,649 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address\n', '2023-06-02 17:20:56,649 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://namenode:9000\n', '2023-06-02 17:20:56,899 [main] INF

In [15]:
run_in_hive("hive -f /data/master_volume/hive_scripts/steam_01_combine.hql")

(['570\tDota 2\tValve\tValve\t\t1456697\t291673\t0\t100,000,000 .. 200,000,000\t39576\t1788\t1021\t944\t0\t0\t0\t612293\n',
  '730\tCounter-Strike: Global Offensive\tValve, Hidden Path Entertainment\tValve\t\t5680980\t756795\t0\t50,000,000 .. 100,000,000\t29878\t858\t6437\t300\t0\t0\t0\t679488\n',
  '578080\tPUBG: BATTLEGROUNDS\tKRAFTON, Inc.\tKRAFTON, Inc.\t\t1139437\t888720\t0\t50,000,000 .. 100,000,000\t21370\t964\t6632\t176\t0\t0\t0\t279181\n',
  '1063730\tNew World\tAmazon Games\tAmazon Games\t\t153807\t73148\t0\t50,000,000 .. 100,000,000\t8392\t590\t3408\t490\t3999\t3999\t0\t20403\n',
  '440\tTeam Fortress 2\tValve\tValve\t\t813122\t56073\t0\t50,000,000 .. 100,000,000\t11858\t1769\t348\t367\t0\t0\t0\t82571\n',
  '304930\tUnturned\tSmartly Dressed Games\tSmartly Dressed Games\t\t438411\t40925\t0\t20,000,000 .. 50,000,000\t10750\t3927\t331\t817\t0\t0\t0\t50423\n',
  '271590\tGrand Theft Auto V\tRockstar North\tRockstar Games\t\t1144201\t208030\t0\t20,000,000 .. 50,000,000\t13182\t8

In [15]:
run_in_hive("hive -f /data/master_volume/hive_scripts/steam_01_combine.hql")

(['570\tDota 2\tValve\tValve\t\t1456697\t291673\t0\t100,000,000 .. 200,000,000\t39576\t1788\t1021\t944\t0\t0\t0\t612293\n',
  '730\tCounter-Strike: Global Offensive\tValve, Hidden Path Entertainment\tValve\t\t5680980\t756795\t0\t50,000,000 .. 100,000,000\t29878\t858\t6437\t300\t0\t0\t0\t679488\n',
  '578080\tPUBG: BATTLEGROUNDS\tKRAFTON, Inc.\tKRAFTON, Inc.\t\t1139437\t888720\t0\t50,000,000 .. 100,000,000\t21370\t964\t6632\t176\t0\t0\t0\t279181\n',
  '1063730\tNew World\tAmazon Games\tAmazon Games\t\t153807\t73148\t0\t50,000,000 .. 100,000,000\t8392\t590\t3408\t490\t3999\t3999\t0\t20403\n',
  '440\tTeam Fortress 2\tValve\tValve\t\t813122\t56073\t0\t50,000,000 .. 100,000,000\t11858\t1769\t348\t367\t0\t0\t0\t82571\n',
  '304930\tUnturned\tSmartly Dressed Games\tSmartly Dressed Games\t\t438411\t40925\t0\t20,000,000 .. 50,000,000\t10750\t3927\t331\t817\t0\t0\t0\t50423\n',
  '271590\tGrand Theft Auto V\tRockstar North\tRockstar Games\t\t1144201\t208030\t0\t20,000,000 .. 50,000,000\t13182\t8